In [1]:
import sys
import sqlite3
from PyQt6.QtWidgets import QApplication, QWidget, QVBoxLayout, QLabel, QLineEdit, QPushButton, QMessageBox, QTableWidget, QTableWidgetItem, QHBoxLayout
from PyQt6.QtCore import QDateTime

class KelioniuRegistratorius(QWidget):
    def __init__(self):
        super().__init__()

        
        self.conn = sqlite3.connect("kelioniu_duomenys.db")

        self.cursor = self.conn.cursor()

        
        self.cursor.execute('''CREATE TABLE IF NOT EXISTS keliones (
                                id INTEGER PRIMARY KEY AUTOINCREMENT,
                                VARDAS TEXT,
                                PAVARDE TEXT,
                                el_pastas TEXT,
                                TELEFONO_NUMERIS TEXT,
                                ISVYKIMO_VIETA TEXT,
                                ATVYKIMO_VIETA TEXT,
                                SUKURIMO_DATA TEXT,
                                ATNAUJINIMO_DATA TEXT
                                )''')
        self.conn.commit()

        
        self.setWindowTitle('Kelionių Registratorius')
        self.setGeometry(200, 200, 800, 600)
        
        self.layout = QVBoxLayout()
        
        
        self.vardas_label = QLabel('VARDAS:')
        self.vardas_input = QLineEdit()
        
        self.pavarde_label = QLabel('PAVARDĖ:')
        self.pavarde_input = QLineEdit()
        
        self.el_pastas_label = QLabel('EL. PAŠTAS:')
        self.el_pastas_input = QLineEdit()
        
        self.telefono_label = QLabel('TELEFONO NUMERIS:')
        self.telefono_input = QLineEdit()
        
        self.isvykimo_vieta_label = QLabel('IŠVYKIMO VIETA:')
        self.isvykimo_vieta_input = QLineEdit()
        
        self.atvykimo_vieta_label = QLabel('ATVYKIMO VIETA:')
        self.atvykimo_vieta_input = QLineEdit()

        
        self.prideti_mygtukas = QPushButton('PRIDĖTI KELIONĘ')

        self.atnaujinti_mygtukas = QPushButton('ATNAUJINTI KELIONĘ')

        self.trinti_mygtukas = QPushButton('IŠTRINTI KELIONĘ')

        
        self.table = QTableWidget()

        self.table.setColumnCount(9)

        self.table.setHorizontalHeaderLabels(["Eil.Nr.", "Vardas", "Pavardė", "El. paštas", "Telefono numeris", 
                                              "Išvykimo vieta", "Atvykimo vieta", "Sukurimo data", "Atnaujinimo data"])
        
        
        self.prideti_mygtukas.clicked.connect(self.prideti_kelione)
        
        self.atnaujinti_mygtukas.clicked.connect(self.atnaujinti_kelione)
        
        self.trinti_mygtukas.clicked.connect(self.trinti_kelione)

        
        self.layout.addWidget(self.vardas_label)
        self.layout.addWidget(self.vardas_input)
        self.layout.addWidget(self.pavarde_label)
        self.layout.addWidget(self.pavarde_input)
        self.layout.addWidget(self.el_pastas_label)
        self.layout.addWidget(self.el_pastas_input)
        self.layout.addWidget(self.telefono_label)
        self.layout.addWidget(self.telefono_input)
        self.layout.addWidget(self.isvykimo_vieta_label)
        self.layout.addWidget(self.isvykimo_vieta_input)
        self.layout.addWidget(self.atvykimo_vieta_label)
        self.layout.addWidget(self.atvykimo_vieta_input)


        
        self.layout.addWidget(self.prideti_mygtukas)
        self.layout.addWidget(self.atnaujinti_mygtukas)
        self.layout.addWidget(self.trinti_mygtukas)


        
        self.layout.addWidget(self.table)


        
        self.setLayout(self.layout)
        self.nustatyti_lentele()
    

    def prideti_kelione(self):
        vardas = self.vardas_input.text()
        pavarde = self.pavarde_input.text()
        el_pastas = self.el_pastas_input.text()
        telefonas = self.telefono_input.text()
        isvykimo_vieta = self.isvykimo_vieta_input.text()
        atvykimo_vieta = self.atvykimo_vieta_input.text()
        sukurimo_data = QDateTime.currentDateTime().toString("yyyy-MM-dd HH:mm:ss")
        atnaujinimo_data = sukurimo_data

        
        self.cursor.execute('''INSERT INTO keliones (vardas, pavarde, el_pastas, telefonas, isvykimo_vieta, 
                            atvykimo_vieta, sukurimo_data, atnaujinimo_data) VALUES (?, ?, ?, ?, ?, ?, ?, ?)''', 
                            (vardas, pavarde, el_pastas, telefonas, isvykimo_vieta, atvykimo_vieta, sukurimo_data, atnaujinimo_data))
        self.conn.commit()
        
        self.rodyti_pranesima("Sėkmė", "Kelionė pridėta sėkmingai!")
        self.nustatyti_lentele()
    
    def atnaujinti_kelione(self):
        
        if self.table.selectedIndexes():
            row = self.table.selectedIndexes()[0].row()
            keliones_id = self.table.item(row, 0).text()
            
            vardas = self.vardas_input.text()
            pavarde = self.pavarde_input.text()
            el_pastas = self.el_pastas_input.text()
            telefonas = self.telefono_input.text()
            isvykimo_vieta = self.isvykimo_vieta_input.text()
            atvykimo_vieta = self.atvykimo_vieta_input.text()
            atnaujinimo_data = QDateTime.currentDateTime().toString("yyyy-MM-dd HH:mm:ss")

            
            self.cursor.execute('''UPDATE keliones SET vardas = ?, pavarde = ?, el_pastas = ?, telefonas = ?, 
                                isvykimo_vieta = ?, atvykimo_vieta = ?, atnaujinimo_data = ? WHERE id = ?''',
                                (vardas, pavarde, el_pastas, telefonas, isvykimo_vieta, atvykimo_vieta, atnaujinimo_data, keliones_id))
            self.conn.commit()
            
            self.rodyti_pranesima("Sėkmė", "Kelionė atnaujinta sėkmingai!")
            self.nustatyti_lentele()
        else:
            self.rodyti_pranesima("Klaida", "Pasirinkite kelionę, kurią norite atnaujinti!")
    
    def trinti_kelione(self):
        
        if self.table.selectedIndexes():
            row = self.table.selectedIndexes()[0].row()
            keliones_id = self.table.item(row, 0).text()

            self.cursor.execute('''DELETE FROM keliones WHERE id = ?''', (keliones_id,))
            self.conn.commit()
            
            self.rodyti_pranesima("Sėkmė", "Kelionė ištrinta sėkmingai!")
            self.nustatyti_lentele()
        else:
            self.rodyti_pranesima("Klaida", "Pasirinkite kelionę, kurią norite ištrinti!")
    
    def nustatyti_lentele(self):
        """Perkraunam visus duomenis į lentelę"""
        self.cursor.execute('SELECT * FROM keliones')
        keliones = self.cursor.fetchall()
        
        self.table.setRowCount(len(keliones))
        
        for row, kelione in enumerate(keliones):
            for col, data in enumerate(kelione):
                self.table.setItem(row, col, QTableWidgetItem(str(data)))
    
    def rodyti_pranesima(self, pavadinimas, pranesimas):
        """Rodome informacinį pranešimą"""
        QMessageBox.information(self, pavadinimas, pranesimas)

if __name__ == '__main__':
    app = QApplication(sys.argv)
    window = KelioniuRegistratorius()
    window.show()
    sys.exit(app.exec())


SystemExit: 0

c:\Users\Vartotojas\2025-03-04 GIT\venv\Lib\site-packages\IPython\core\interactiveshell.py:3557: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
